# This notebook is to evaluate network performance on test shots

In [ ]:
# imports
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
import os

In [ ]:
# let's first see the shots
SHOTS_DIR = 'test_shots'
# list all the files inside
shot_files = os.listdir(SHOTS_DIR)
names = [f.split('.')[0] for f in shot_files if f.endswith('.mat')]
print(f'Found {len(names)} shots: {names}')

for shot in shot_files:
    d = loadmat(os.path.join(SHOTS_DIR, shot))
    vars = [k for k in d.keys() if not k.startswith('__')]
    # for each var, print the dimensions
    for var in vars: print(f'{var}: {d[var].shape}')